# RedisVectorStore

This notebook demonstrates the usage of RedisVectorStore from the langchain-redis package. RedisVectorStore leverages Redis as a vector database, enabling efficient storage, retrieval, and similarity search of vector embeddings.

## Installation

First, we need to install the necessary packages. Run the following command to install langchain-redis, sentence-transformers, and scikit-learn:

In [1]:
%pip install -U langchain-redis langchain-huggingface sentence-transformers scikit-learn

Note: you may need to restart the kernel to use updated packages.


## Importing Required Libraries
We'll import the necessary libraries for our tasks:

In [2]:
import os
from langchain_redis import RedisVectorStore
from sklearn.datasets import fetch_20newsgroups
from langchain.docstore.document import Document

## Setting up Redis Connection
To use RedisVectorStore, you need a running Redis instance. For this example, we assume a local Redis instance running on the default port. Modify the URL if your setup differs:

In [3]:
import os

# Use the environment variable if set, otherwise default to localhost
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
print(f"Connecting to Redis at: {REDIS_URL}")

Connecting to Redis at: redis://redis:6379


Let's check that Redis is up an running by pinging it:

In [4]:
import redis

redis_client = redis.from_url(REDIS_URL)
redis_client.ping()

True

## Preparing Sample Data
We'll use a subset of the 20 Newsgroups dataset for this demonstration. This dataset contains newsgroup posts on various topics. We'll focus on two categories: 'alt.atheism' and 'sci.space':

In [5]:
categories = ["alt.atheism", "sci.space"]
newsgroups = fetch_20newsgroups(
    subset="train", categories=categories, shuffle=True, random_state=42
)

# Use only the first 250 documents
texts = newsgroups.data[:250]
metadata = [
    {"category": newsgroups.target_names[target]} for target in newsgroups.target[:250]
]

documents = [
    Document(page_content=text, metadata=meta) for text, meta in zip(texts, metadata)
]
len(documents)

250

Let's inspect the first document:

In [6]:
documents[0]

Document(metadata={'category': 'alt.atheism'}, page_content='From: bil@okcforum.osrhe.edu (Bill Conner)\nSubject: Re: Not the Omni!\nNntp-Posting-Host: okcforum.osrhe.edu\nOrganization: Okcforum Unix Users Group\nX-Newsreader: TIN [version 1.1 PL6]\nLines: 18\n\nCharley Wingate (mangoe@cs.umd.edu) wrote:\n: \n: >> Please enlighten me.  How is omnipotence contradictory?\n: \n: >By definition, all that can occur in the universe is governed by the rules\n: >of nature. Thus god cannot break them. Anything that god does must be allowed\n: >in the rules somewhere. Therefore, omnipotence CANNOT exist! It contradicts\n: >the rules of nature.\n: \n: Obviously, an omnipotent god can change the rules.\n\nWhen you say, "By definition", what exactly is being defined;\ncertainly not omnipotence. You seem to be saying that the "rules of\nnature" are pre-existant somehow, that they not only define nature but\nactually cause it. If that\'s what you mean I\'d like to hear your\nfurther thoughts on the q

## Creating Embeddings
We'll use the SentenceTransformer model to create embeddings. This model runs locally and doesn't require an API key:

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="msmarco-distilbert-base-v4")

/home/jupyter/venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/319 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Basic Usage with LangChain's RedisVectorStore
Now we'll demonstrate basic usage of RedisVectorStore, including creating an instance, inserting data, and performing a simple similarity search.

### Creating a RedisVectorStore instance and inserting data
We'll create a RedisVectorStore instance and populate it with our sample data:

In [8]:
vector_store = RedisVectorStore.from_documents(
    documents,
    embeddings,
    redis_url=REDIS_URL,
    index_name="newsgroups",
    metadata_schema=[
        {"name": "category", "type": "tag"},
    ],
)

### Performing a simple similarity search
Let's perform a basic similarity search using a query about space exploration:

In [9]:
query = "Tell me about space exploration"
results = vector_store.similarity_search(query, k=2)

print("Simple Similarity Search Results:")
for doc in results:
    print(f"Content: {doc.page_content[:100]}...")
    print(f"Metadata: {doc.metadata}")
    print()

Simple Similarity Search Results:
Content: From: aa429@freenet.carleton.ca (Terry Ford)
Subject: A flawed propulsion system: Space Shuttle
X-Ad...
Metadata: {'category': 'sci.space'}

Content: From: nsmca@aurora.alaska.edu
Subject: Space Design Movies?
Article-I.D.: aurora.1993Apr23.124722.1
...
Metadata: {'category': 'sci.space'}



## Advanced Queries with RedisVectorStore
RedisVectorStore supports more advanced query types. We'll demonstrate similarity search with metadata filtering, maximum marginal relevance search, and similarity search with score.

### Similarity search with metadata filtering
We can filter our search results based on metadata:

In [10]:
from redisvl.query.filter import Tag

query = "Tell me about space exploration"

# Create a filter expression
filter_condition = Tag("category") == "sci.space"

filtered_results = vector_store.similarity_search(query, k=2, filter=filter_condition)

print("Filtered Similarity Search Results:")
for doc in filtered_results:
    print(f"Content: {doc.page_content[:100]}...")
    print(f"Metadata: {doc.metadata}")
    print()

Filtered Similarity Search Results:
Content: From: aa429@freenet.carleton.ca (Terry Ford)
Subject: A flawed propulsion system: Space Shuttle
X-Ad...
Metadata: {'category': 'sci.space'}

Content: From: nsmca@aurora.alaska.edu
Subject: Space Design Movies?
Article-I.D.: aurora.1993Apr23.124722.1
...
Metadata: {'category': 'sci.space'}



### Maximum marginal relevance search
Maximum marginal relevance search helps in getting diverse results:

In [11]:
# Maximum marginal relevance search with filter
mmr_results = vector_store.max_marginal_relevance_search(
    query, k=2, fetch_k=10, filter=filter_condition
)

print("Maximum Marginal Relevance Search Results:")
for doc in mmr_results:
    print(f"Content: {doc.page_content[:100]}...")
    print(f"Metadata: {doc.metadata}")
    print()

Maximum Marginal Relevance Search Results:
Content: From: aa429@freenet.carleton.ca (Terry Ford)
Subject: A flawed propulsion system: Space Shuttle
X-Ad...
Metadata: {'category': 'sci.space'}

Content: From: moroney@world.std.com (Michael Moroney)
Subject: Re: Vulcan? (No, not the guy with the ears!)
...
Metadata: {'category': 'sci.space'}



### Similarity search with score
We can also get similarity scores along with our search results:

In [12]:
# Similarity search with score and filter
scored_results = vector_store.similarity_search_with_score(
    query, k=2, filter=filter_condition
)

print("Similarity Search with Score Results:")
for doc, score in scored_results:
    print(f"Content: {doc.page_content[:100]}...")
    print(f"Metadata: {doc.metadata}")
    print(f"Score: {score}")
    print()

Similarity Search with Score Results:
Content: From: aa429@freenet.carleton.ca (Terry Ford)
Subject: A flawed propulsion system: Space Shuttle
X-Ad...
Metadata: {'category': 'sci.space'}
Score: 0.569670796394

Content: From: nsmca@aurora.alaska.edu
Subject: Space Design Movies?
Article-I.D.: aurora.1993Apr23.124722.1
...
Metadata: {'category': 'sci.space'}
Score: 0.590400338173



## Cleanup
After we're done, it's important to clean up our Redis indices:

In [13]:
# Delete the underlying index and it's data
vector_store.index.delete(drop=True)